<a href="https://colab.research.google.com/github/Adityaraj0021/Capstone/blob/main/Final_Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import ee
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, f1_score
from matplotlib.colors import ListedColormap
import folium
import geopandas as gpd
from datetime import datetime

In [2]:
# Initialize Earth Engine
try:
    ee.Initialize()
except Exception:
    ee.Authenticate()
    ee.Initialize(project='ee-raj282251')

In [3]:
# Define Ludhiana district as the area of interest
ludhiana = ee.FeatureCollection('FAO/GAUL/2015/level2') \
    .filter(ee.Filter.And(
        ee.Filter.eq('ADM1_NAME', 'Punjab'),
        ee.Filter.eq('ADM2_NAME', 'Ludhiana')
    ))

In [4]:
# Extract the geometry
aoi = ludhiana.geometry()

In [5]:
# Create a map centered on Ludhiana
Map = geemap.Map()
Map.centerObject(aoi, 10)
Map.addLayer(aoi, {}, 'Ludhiana District')

In [6]:
# Define LULC classes with detailed descriptions
lulc_classes = {
    0: {'name': 'Urban/Built-up', 'color': 'red', 'description': 'Buildings, roads, and other man-made structures'},
    1: {'name': 'Dense Vegetation', 'color': 'darkgreen', 'description': 'Forests, parks, and dense vegetation areas'},
    2: {'name': 'Water', 'color': 'blue', 'description': 'Rivers, lakes, ponds, and water bodies'},
    3: {'name': 'Agriculture', 'color': 'yellow', 'description': 'Cropland and agricultural fields'},
    4: {'name': 'Barren Land', 'color': 'brown', 'description': 'Bare soil, sand, and areas with sparse vegetation'},
    5: {'name': 'Sparse Vegetation', 'color': 'lightgreen', 'description': 'Areas with partial and scattered vegetation'},
    6: {'name': 'Wetlands', 'color': 'cyan', 'description': 'Marshes and areas with waterlogged soil'}
}

In [7]:
def maskS2clouds(image):
    # Check if QA60 band exists (some S2_SR images may not have it)
    bandNames = image.bandNames()
    condition = bandNames.contains('QA60')

    # Only apply cloud mask if QA60 exists, otherwise return the image without cloud masking
    def apply_mask(img):
        qa = img.select('QA60')
        cloudBitMask = 1 << 10  # Bit 10 = clouds
        mask = qa.bitwiseAnd(cloudBitMask).eq(0)  # Only mask clouds
        return img.updateMask(mask).divide(10000)  # Rescale the image by dividing by 10000

    # If QA60 exists, apply the mask, else return the original image
    return ee.Algorithms.If(condition, apply_mask(image), image.divide(10000))


In [8]:
# Get multi-seasonal Sentinel-2 collection for better classification
seasons = [
    {'name': 'Winter', 'start': '2023-11-01', 'end': '2023-12-31'},
    {'name': 'Pre-monsoon', 'start': '2023-04-01', 'end': '2023-05-31'},
    {'name': 'Post-monsoon', 'start': '2023-09-01', 'end': '2023-10-31'},
    {'name': 'Monsoon', 'start': '2023-07-01', 'end': '2023-08-31'}
]

In [9]:
# Create seasonal composites
seasonal_composites = {}
for season in seasons:
    s2_collection = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterDate(season['start'], season['end'])
        .filterBounds(aoi)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
        .map(maskS2clouds))

    if s2_collection.size().getInfo() > 0:
        seasonal_composites[season['name']] = s2_collection.median().clip(aoi)
        print(f"Created {season['name']} composite with {s2_collection.size().getInfo()} images")
    else:
        print(f"No images available for {season['name']}")

Created Winter composite with 60 images
Created Pre-monsoon composite with 64 images
Created Post-monsoon composite with 69 images
Created Monsoon composite with 23 images


In [10]:
# Use winter composite as primary if available, otherwise use the first available composite
composite = None
for season in seasons:
    if season['name'] in seasonal_composites:
        composite = seasonal_composites[season['name']]
        primary_season = season['name']
        break

if composite is None:
    raise Exception("No imagery available for any season")
else:
    print(f"Using {primary_season} as primary composite")

Using Winter as primary composite


In [11]:
def calculate_indices(img, season_suffix=''):
    ndvi = img.normalizedDifference(['B8', 'B4']).rename(f'NDVI{season_suffix}')
    ndwi = img.normalizedDifference(['B3', 'B8']).rename(f'NDWI{season_suffix}')
    ndbi = img.normalizedDifference(['B11', 'B8']).rename(f'NDBI{season_suffix}')
    mndwi = img.normalizedDifference(['B3', 'B11']).rename(f'MNDWI{season_suffix}')

    savi = img.expression(
        '((NIR - RED) / (NIR + RED + L)) * (1 + L)', {
            'NIR': img.select('B8'),
            'RED': img.select('B4'),
            'L': 0.5
        }).rename(f'SAVI{season_suffix}')

    evi = img.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': img.select('B8'),
            'RED': img.select('B4'),
            'BLUE': img.select('B2')
        }).rename(f'EVI{season_suffix}')

    return img.addBands([ndvi, ndwi, ndbi, mndwi, savi, evi])

# Process seasonal composites
seasonal_composites = {}

for season in seasons:
    s2_collection = (ee.ImageCollection('COPERNICUS/S2_SR')
                     .filterDate(season['start'], season['end'])
                     .filterBounds(aoi)
                     .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
                     .map(maskS2clouds))

    if s2_collection.size().getInfo() > 0:
        composite = s2_collection.median().clip(aoi)
        suffix = f"_{season['name'].lower().replace('-', '_')}"
        seasonal_composites[season['name']] = calculate_indices(composite, season_suffix=suffix)
        print(f"Created {season['name']} composite with {s2_collection.size().getInfo()} images")
    else:
        print(f"No images available for {season['name']}")

# Use winter as primary, fallback to the first available
composite = None
primary_season = ''
for season in seasons:
    if season['name'] in seasonal_composites:
        composite = seasonal_composites[season['name']]
        primary_season = season['name']
        break

if composite is None:
    raise Exception("No imagery available for any season")
else:
    print(f"Using {primary_season} as primary composite")

# Set initial bands for classification
suffix = f"_{primary_season.lower().replace('-', '_')}"
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12',
         f'NDVI{suffix}', f'NDWI{suffix}', f'NDBI{suffix}', f'MNDWI{suffix}', f'SAVI{suffix}', f'EVI{suffix}']

# Add key bands from other seasons
for season in seasonal_composites:
    if season != primary_season:
        season_suffix = f"_{season.lower().replace('-', '_')}"
        for index in ['NDVI', 'NDWI', 'NDBI']:
            band = f'{index}{season_suffix}'
            composite = composite.addBands(seasonal_composites[season].select(band))
            bands.append(band)

# Create map and display
# Map = Map(center=[30.9, 75.85], zoom=8)

Map.addLayer(composite, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}, 'Sentinel-2 RGB')
Map.addLayer(composite, {'bands': ['B8', 'B4', 'B3'], 'min': 0, 'max': 0.3}, 'False Color (NIR)', False)

Map.addLayer(composite.select(f'NDVI{suffix}'), {'min': -0.2, 'max': 0.8, 'palette': ['red', 'yellow', 'green']}, f'NDVI ({primary_season})', False)
Map.addLayer(composite.select(f'NDWI{suffix}'), {'min': -0.5, 'max': 0.5, 'palette': ['brown', 'white', 'blue']}, f'NDWI ({primary_season})', False)
Map.addLayer(composite.select(f'NDBI{suffix}'), {'min': -0.5, 'max': 0.5, 'palette': ['green', 'white', 'red']}, f'NDBI ({primary_season})', False)
Map.addLayer(composite.select(f'MNDWI{suffix}'), {'min': -0.5, 'max': 0.5, 'palette': ['brown', 'white', 'cyan']}, f'MNDWI ({primary_season})', False)

Map

Created Winter composite with 60 images
Created Pre-monsoon composite with 64 images
Created Post-monsoon composite with 69 images
Created Monsoon composite with 23 images
Using Winter as primary composite


Map(center=[30.8108318833292, 75.82406946610111], controls=(WidgetControl(options=['position', 'transparent_bg…

In [12]:
# Define LULC class labels
# lulc_classes = {
#     0: {"name": "Urban", "color": "red", "description": "Built-up areas"},
#     1: {"name": "Dense Vegetation", "color": "green", "description": "Forest, dense croplands"},
#     2: {"name": "Water", "color": "blue", "description": "Rivers, lakes, canals"},
#     3: {"name": "Agriculture", "color": "yellow", "description": "Crops, farmlands"},
#     4: {"name": "Barren", "color": "gray", "description": "Bare soil or sparsely covered"},
#     5: {"name": "Sparse Vegetation", "color": "lightgreen", "description": "Grasslands, scrub"},
#     6: {"name": "Wetlands", "color": "cyan", "description": "Swamps, marshes"},
# }


def generate_training_points(composite, aoi, num_points_per_class=100, index_suffix=''):
    """
    Generate training points for each LULC class using refined spectral thresholds.

    Args:
        composite (ee.Image): The multi-band image with spectral indices.
        aoi (ee.Geometry): The area of interest for sampling.
        num_points_per_class (int): Number of training points per class.
        index_suffix (str): The suffix to append to band names, e.g., "_winter".

    Returns:
        ee.FeatureCollection: FeatureCollection of training points with class labels.
    """

    # Define full band names with suffix
    ndvi = f'NDVI{index_suffix}'
    ndwi = f'NDWI{index_suffix}'
    ndbi = f'NDBI{index_suffix}'
    mndwi = f'MNDWI{index_suffix}'
    savi = f'SAVI{index_suffix}'
    evi = f'EVI{index_suffix}'

    # Define class masks
    class_masks = {
        0: composite.select(ndbi).gt(0.0)
            .And(composite.select(ndvi).lt(0.3))
            .And(composite.select(mndwi).lt(-0.1)),

        1: composite.select(ndvi).gt(0.6)
            .And(composite.select(evi).gt(0.4)),

        2: composite.select(mndwi).gt(0.2)
            .And(composite.select(ndwi).gt(0.1))
            .And(composite.select(ndvi).lt(0.1)),

        3: composite.select(ndvi).gt(0.3)
            .And(composite.select(ndvi).lt(0.6))
            .And(composite.select(ndbi).lt(0))
            .And(composite.select(savi).gt(0.2)),

        4: composite.select(ndvi).lt(0.2)
            .And(composite.select(ndbi).lt(0.1))
            .And(composite.select(ndbi).gt(-0.1))
            .And(composite.select(mndwi).lt(0)),

        5: composite.select(ndvi).gt(0.2)
            .And(composite.select(ndvi).lt(0.4))
            .And(composite.select(savi).lt(0.3)),

        6: composite.select(mndwi).gt(0)
            .And(composite.select(mndwi).lt(0.2))
            .And(composite.select(ndvi).gt(0.2))
            .And(composite.select(ndvi).lt(0.4))
    }

    # Initialize empty collection
    all_points = ee.FeatureCollection([])

    # Loop through each class and generate samples
    for class_id in lulc_classes:
        if class_id not in class_masks:
            continue

        # Clean up mask using morphological operations
        mask = class_masks[class_id].selfMask()
        cleaned_mask = mask.focalMin(30, 'circle', 'meters').focalMax(15, 'circle', 'meters')

        # Sample training points
        points = cleaned_mask.sample(
            region=aoi,
            scale=10,
            numPixels=num_points_per_class,
            seed=42 + class_id,
            geometries=True
        ).map(lambda f: f.set('class', class_id))

        all_points = all_points.merge(points)

    return all_points


In [13]:
# Use the correct suffix from the selected primary composite
suffix = f"_{primary_season.lower().replace('-', '_')}"

# Generate suggested training points
suggested_points = generate_training_points(composite, aoi, num_points_per_class=150, index_suffix=suffix)


In [14]:
# import geemap
# import ee
from matplotlib import colors as mcolors

def hex_to_rgb_tuple(color_str):
    try:
        rgb = mcolors.to_rgb(color_str)  # values in 0-1 range
        return tuple(int(c * 255) for c in rgb)
    except:
        return (0, 0, 0)  # fallback: black if invalid

def create_verification_interface(composite, suggested_points, aoi):
    # Create interactive map
    verification_map = geemap.Map()
    verification_map.centerObject(aoi, 10)

    # Add base layers
    verification_map.add_basemap('HYBRID')
    verification_map.addLayer(composite, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}, 'Sentinel-2 RGB')
    verification_map.addLayer(composite, {'bands': ['B8', 'B4', 'B3'], 'min': 0, 'max': 0.3}, 'False Color (NIR)', False)

    # Add spectral indices
    verification_map.addLayer(
        composite.select('NDVI_monsoon'),
        {'min': -0.2, 'max': 0.8, 'palette': ['red', 'yellow', 'green']},
        'NDVI (Monsoon)', False
    )
    verification_map.addLayer(
        composite.select('NDWI_monsoon'),
        {'min': -0.5, 'max': 0.5, 'palette': ['brown', 'white', 'blue']},
        'NDWI (Monsoon)', False
    )
    verification_map.addLayer(
        composite.select('NDBI_monsoon'),
        {'min': -0.5, 'max': 0.5, 'palette': ['green', 'white', 'red']},
        'NDBI (Monsoon)', False
    )
    verification_map.addLayer(
        composite.select('MNDWI_winter'),
        {'min': -0.5, 'max': 0.5, 'palette': ['brown', 'white', 'cyan']},
        'MNDWI (Winter)', False
    )

    # Add styled suggested points and prepare legend
    legend_keys = []
    legend_colors = []

    for class_id, class_info in lulc_classes.items():
        class_points = suggested_points.filter(ee.Filter.eq('class', class_id))
        name = class_info['name']
        color = class_info['color']

        if isinstance(color, (list, tuple)):
            color = color[0]
        color = str(color)

        verification_map.addLayer(class_points, {'color': color}, f"{name} Points")

        legend_keys.append(name)
        legend_colors.append(hex_to_rgb_tuple(color))  # convert to RGB tuple

    # Add tools
    verification_map.add_draw_control()
    verification_map.add_inspector()
    verification_map.set_options(minZoom=9, maxZoom=16)

    # Add legend
    verification_map.add_legend(
        title="LULC Classes",
        keys=legend_keys,
        colors=legend_colors
    )

    return verification_map


In [15]:
verification_map = create_verification_interface(composite, suggested_points, aoi)
display(verification_map)


Map(center=[30.81083188332916, 75.82406946610104], controls=(WidgetControl(options=['position', 'transparent_b…

In [16]:
import time

def stratified_data_collection_fast2(composite, aoi, classes, points_per_strata=5, strata_per_class=2):
    """Fast version: Use unsupervised clustering to identify diverse areas within each class"""
    start_time = time.time()
    print("⏳ Starting stratified data collection...")

    # Define bands
    cluster_bands = ['B2', 'B3', 'B4', 'B8', 'B11', 'NDVI_monsoon', 'NDBI_monsoon', 'NDWI_monsoon', 'MNDWI_winter']
    cluster_image = composite.select(cluster_bands)

    # Normalize bands
    means = cluster_image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=30,
        maxPixels=1e9
    ).getInfo()

    stds = cluster_image.reduceRegion(
        reducer=ee.Reducer.stdDev(),
        geometry=aoi,
        scale=30,
        maxPixels=1e9
    ).getInfo()

    normalized_bands = []
    for band in cluster_bands:
        norm = cluster_image.select(band).subtract(means[band]).divide(stds[band])
        normalized_bands.append(norm)

    normalized_image = ee.Image.cat(normalized_bands).rename(cluster_bands)

    # Clustering
    n_clusters = strata_per_class * len(classes)
    print(f"🔢 Running KMeans with {n_clusters} clusters...")

    training_points = normalized_image.sample(
        region=aoi,
        scale=30,
        numPixels=2000,  # Reduced for speed
        seed=42,
        geometries=False
    )

    clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training_points)
    clustered = normalized_image.cluster(clusterer).rename('cluster')

    # Map preview
    Map.addLayer(clustered.randomVisualizer(), {}, 'Clustered Preview')

    # Point collection
    all_strata_points = ee.FeatureCollection([])

    for class_id, class_info in classes.items():
        print(f"\n📦 Class {class_id}: {class_info['name']}")

        # Define masks
        # Class mask using monsoon indices
        if class_id == 0:
            class_mask = composite.select('NDBI_monsoon').gt(0).And(composite.select('NDVI_monsoon').lt(0.3))
        elif class_id == 1:
            class_mask = composite.select('NDVI_monsoon').gt(0.6)
        elif class_id == 2:
            class_mask = composite.select('MNDWI_winter').gt(0.2)
        elif class_id == 3:
            class_mask = composite.select('NDVI_monsoon').gt(0.3).And(composite.select('NDVI_monsoon').lt(0.6))
        elif class_id == 4:
            class_mask = composite.select('NDVI_monsoon').lt(0.2).And(composite.select('MNDWI_winter').lt(0))
        elif class_id == 5:
            class_mask = composite.select('NDVI_monsoon').gt(0.2).And(composite.select('NDVI_monsoon').lt(0.4))
        elif class_id == 6:
            class_mask = composite.select('MNDWI_winter').gt(0).And(composite.select('NDVI_monsoon').gt(0.2))
        else:
            print(f"⚠️ Skipping class {class_id}")
            continue

        class_mask = class_mask.selfMask().focalMin(20, 'circle', 'meters')

        for i in range(n_clusters):
            print(f"    • Sampling class {class_id}, cluster {i}...")
            cluster_mask = clustered.eq(i)
            combined_mask = class_mask.And(cluster_mask)

            try:
                strata_points = combined_mask.sample(
                    region=aoi,
                    scale=10,
                    numPixels=points_per_strata,
                    seed=42 + class_id * 100 + i,
                    geometries=True
                ).map(lambda f: f.set({
                    'class': class_id,
                    'strata': i,
                    'classname': class_info['name']
                }))

                # Skip size().getInfo() for speed
                all_strata_points = all_strata_points.merge(strata_points)

            except Exception as e:
                print(f"⚠️ Error in class {class_id}, cluster {i}: {e}")

    print(f"\n✅ Done! Total time: {round(time.time() - start_time, 2)} seconds.")
    return all_strata_points


In [17]:
stratified_points = stratified_data_collection_fast2(composite, aoi, lulc_classes)


⏳ Starting stratified data collection...
🔢 Running KMeans with 14 clusters...

📦 Class 0: Urban/Built-up
    • Sampling class 0, cluster 0...
    • Sampling class 0, cluster 1...
    • Sampling class 0, cluster 2...
    • Sampling class 0, cluster 3...
    • Sampling class 0, cluster 4...
    • Sampling class 0, cluster 5...
    • Sampling class 0, cluster 6...
    • Sampling class 0, cluster 7...
    • Sampling class 0, cluster 8...
    • Sampling class 0, cluster 9...
    • Sampling class 0, cluster 10...
    • Sampling class 0, cluster 11...
    • Sampling class 0, cluster 12...
    • Sampling class 0, cluster 13...

📦 Class 1: Dense Vegetation
    • Sampling class 1, cluster 0...
    • Sampling class 1, cluster 1...
    • Sampling class 1, cluster 2...
    • Sampling class 1, cluster 3...
    • Sampling class 1, cluster 4...
    • Sampling class 1, cluster 5...
    • Sampling class 1, cluster 6...
    • Sampling class 1, cluster 7...
    • Sampling class 1, cluster 8...
    • Sampl

In [18]:
# Merge both types of points
combined_points = suggested_points.merge(stratified_points)

In [19]:
task = ee.batch.Export.table.toDrive(
    collection=combined_points,
    description='Combined_Sample_Points_GeoJSON',
    folder='GEE_Exports',  # You can specify a folder in your Google Drive
    fileFormat='GeoJSON'   # Export as GeoJSON format
)
task.start()


In [20]:
# Print out all the available bands
print(composite.bandNames().getInfo())


['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60', 'NDVI_winter', 'NDWI_winter', 'NDBI_winter', 'MNDWI_winter', 'SAVI_winter', 'EVI_winter', 'NDVI_pre_monsoon', 'NDWI_pre_monsoon', 'NDBI_pre_monsoon', 'NDVI_post_monsoon', 'NDWI_post_monsoon', 'NDBI_post_monsoon', 'NDVI_monsoon', 'NDWI_monsoon', 'NDBI_monsoon']


In [21]:
# /content/drive/MyDrive/GEE_Exports/Combined_Sample_Points_GeoJSON.geojson

In [22]:
import geopandas as gpd
from sklearn.model_selection import train_test_split

# Load your labeled sample points
gdf = gpd.read_file("/content/drive/MyDrive/GEE_Exports/Combined_Sample_Points_GeoJSON.geojson")
print(gdf.head())



                  id  MNDWI_winter  NDBI_monsoon  NDBI_winter  NDVI_monsoon  \
0  1_1_1_1_1_1_1_2_0           NaN           NaN          1.0           NaN   
1  1_1_1_1_1_1_1_2_1           NaN           NaN          1.0           NaN   
2  1_1_1_1_1_1_1_2_2           NaN           NaN          1.0           NaN   
3  1_1_1_1_1_1_1_2_3           NaN           NaN          1.0           NaN   
4  1_1_1_1_1_1_1_2_4           NaN           NaN          1.0           NaN   

   NDVI_winter  class classname  strata                   geometry  
0          NaN      0      None     NaN    POINT (75.7686 30.9089)  
1          NaN      0      None     NaN   POINT (75.61262 30.6741)  
2          NaN      0      None     NaN   POINT (75.96603 30.9028)  
3          NaN      0      None     NaN  POINT (75.46559 30.66341)  
4          NaN      0      None     NaN  POINT (76.10722 30.97341)  


In [23]:
# Shape gives (rows, columns)
print("Shape:", gdf.shape)

# Size gives total number of elements (rows × columns)
print("Total size (elements):", gdf.size)


Shape: (603, 10)
Total size (elements): 6030


In [24]:
# Count total NaNs per column
nan_counts = gdf.isna().sum()

# Count total "None" strings if they exist as strings (not Python NoneType)
none_counts = (gdf == 'None').sum()

# Combine both for overview
total_missing = nan_counts + none_counts

print("Missing values summary:")
print(total_missing)


Missing values summary:
id                0
MNDWI_winter    599
NDBI_monsoon    595
NDBI_winter     479
NDVI_monsoon    470
NDVI_winter     269
class             0
classname       461
strata          461
geometry          0
dtype: int64


In [25]:
# Define features and labels
features = ['B2', 'B3', 'B4', 'B8', 'B11', 'NDVI_monsoon', 'NDWI_monsoon', 'NDBI_monsoon', 'MNDWI_winter']
X = gdf[features]
y = gdf['class']  # Or whatever your class label column is called

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42, test_size=0.3)


KeyError: "['B2', 'B3', 'B4', 'B8', 'B11', 'NDWI_monsoon'] not in index"